In [5]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle
import re
import requests
import PIL
from PIL import Image
from io import BytesIO
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import os
from google.colab import drive

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('train.csv',engine='python')
df.head()

,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49


In [ ]:
import re
import requests
import PIL
from PIL import Image
from io import BytesIO

In [ ]:
def open_image(url: str) -> None:
    """
    Downloads an image from a given URL and displays it.

    Args:
        url (str): The public URL of the image.
    """
    print(f"Attempting to open image from: {url}")
    try:
        # Download the image content
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        content_type = response.headers.get('Content-Type', '').lower()
        if not content_type.startswith('image/'):
            print("Error: URL did not return image data.")
            return

        # Open and display the image from memory
        img = Image.open(BytesIO(response.content))
        img.show()
        print(f"Image opened successfully.")

    except requests.RequestException as e:
        # Catches connection, HTTP status, and timeout errors
        print(f"Failed to download image: {e}")
    except IOError:
        # Catches PIL error if the downloaded data isn't a valid image
        print("Error: Downloaded data is not a valid image file.")

In [ ]:
import pandas as pd
import re

def extract_pack_count(title):
    """
    Attempts to extract pack count from various formats in the title string.
    Priority: 1. (Pack of X) | 2. X-Pack/X Pack/X Count | 3. None (NaN)

    Args:
        title (str): The raw product title string.

    Returns:
        str or None: The extracted pack count number as a string, or None if not found.
    """
    if pd.isna(title):
        return None

    # 1. Look for parenthetical formats (e.g., (Pack of 6)) - highest priority
    match_parenthetical = re.search(r'\((?:Pack|Case|Set|Ct|Count)\s*(?:of\s*)?(\d+)\)', title, re.IGNORECASE)
    if match_parenthetical:
        return match_parenthetical.group(1)

    # 2. Look for hyphenated/spaced formats (e.g., 6-Pack or 6 Pack or 12 Cases)
    match_hyphenated = re.search(r'(\d+)\s*[-]?\s*(?:Pack|Packs|Case|Cases|Set|Sets|Count|Counts)', title, re.IGNORECASE)
    if match_hyphenated:
        return match_hyphenated.group(1)

    return None


def extract_catalog_info(df):
    """
    Extracts structured information (Brand, Core Product Name, Size/Weight, Unit, Pack Count)
    from the 'catalog_content' column of a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing the 'catalog_content' column.

    Returns:
        pd.DataFrame: DataFrame with new extracted columns, including the full catalog content.
    """
    # Create a copy of the full catalog content to ensure it is not lost,
    # and rename the original column to avoid confusion with the interim title string.
    df['Catalog_Content_Full'] = df['catalog_content']


    # 1. Prepare the Title String for Extraction (Truncate descriptive parts)
    # The extraction process only needs the first line (Item Name)
    df['Extraction_Title'] = (
        df['catalog_content']
        .str.replace(r'^Item Name:\s*', '', regex=True)
        .str.split(r'\\n', n=1, expand=True)[0] # Use newline \n to isolate the first line
        .str.strip()
    )


    # 2. Extract Size/Weight and Unit
    size_unit_pattern = r'(\d+\.?\d*)\s*[-]?\s*(Ounces?|oz|case|Fl Oz|L|ml|g|lb)'
    df['Size_Value'] = df['Extraction_Title'].str.extract(size_unit_pattern, flags=re.IGNORECASE).iloc[:, 0]
    df['Unit'] = df['Extraction_Title'].str.extract(size_unit_pattern, flags=re.IGNORECASE).iloc[:, 1]

    # 3. Extract Pack Count (Uses helper function for robust matching)
    df['Pack_Count'] = df['Extraction_Title'].apply(extract_pack_count)


    # 4. Initial Cleanup: Remove size/pack info from the Title (simplifies brand separation)
    clean_title_pattern = r',\s*\d+\.?\d*\s*[-]?\s*(Ounces?|oz|Fl Oz|L|ml|g|lb|per case)(\s*\(Pack of\s*\d+\))?\.?$'
    df['Product_Title_Interim'] = df['Extraction_Title'].str.replace(clean_title_pattern, '', regex=True).str.strip()


    # 5. Separate Brand Name and Core Product Name (Heuristic)
    brand_core_pattern = r'^([A-Z0-9\'-]+(?:\s[A-Z0-9\'-]+)?)\s*(.*)'
    extracted_parts = df['Product_Title_Interim'].str.extract(brand_core_pattern, flags=re.IGNORECASE)

    df['Brand_Name'] = extracted_parts[0].fillna(df['Product_Title_Interim']).str.strip()
    df['Product_Name_Core'] = extracted_parts[1].fillna('').str.strip()


    # 5.5. Surgical IPQ Cleanup (Removes remaining size/pack info from core name)
    all_ipq_pattern = (
        r'(\s*\(Pack of\s*\d+\))'
        r'|(\s*[-]?\s*\d+\s*[-]?\s*(?:Pack|Packs|Case|Cases|Set|Sets|Count))'
        r'|(\s*[-]?\s*\d+\.?\d*\s*[-]?\s*(?:Ounces?|oz|Fl Oz|L|ml|g|lb))'
        r'|(\s*,\s*per case)'
    )

    df['Product_Name_Core'] = df['Product_Name_Core'].str.replace(
        all_ipq_pattern,
        '',
        regex=True,
        flags=re.IGNORECASE
    ).str.strip(' ,-')


    # 6. Redundancy Check and Final Name Assignment
    def clean_core_name(row):
        brand = row['Brand_Name']
        core = row['Product_Name_Core']

        if pd.notna(core) and pd.notna(brand) and core.lower().startswith(brand.lower()):
            # Remove the brand and subsequent comma/space if present
            cleaned_core = re.sub(f'^{re.escape(brand)}\s*,?\s*', '', core, flags=re.IGNORECASE).strip()
            return cleaned_core

        return core

    df['Product_Name_Clean'] = df.apply(clean_core_name, axis=1)

    # Select and reorder the final columns, INCLUDING the full catalog content for BERT
    df_extracted = df[['sample_id', 'Brand_Name', 'Product_Name_Clean', 'Size_Value', 'Unit', 'Pack_Count', 'Catalog_Content_Full', 'image_link', 'price']]

    return df_extracted


<>:102: SyntaxWarning: invalid escape sequence '\s'
<>:102: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3349014029.py:102: SyntaxWarning: invalid escape sequence '\s'
  cleaned_core = re.sub(f'^{re.escape(brand)}\s*,?\s*', '', core, flags=re.IGNORECASE).strip()


In [ ]:
df_extracted=extract_catalog_info(df)

In [ ]:
df_extracted.head()

,sample_id,Brand_Name,Product_Name_Clean,Size_Value,Unit,Pack_Count,Catalog_Content_Full,image_link,price
0,33127,La Victoria,Green Taco Sauce Mild,12,Ounce,6,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89
1,198967,Salerno Cookies,The Original Butter Cookies,8,Ounce,4,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12
2,261251,Bear Creek,"Hearty Soup Bowl, Creamy Chicken with Rice",1.9,Ounce,6,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97
3,55858,Judee,’s Blue Cheese Powder - Gluten-Free and Nut-Fr...,11.25,oz,None,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34
4,292686,kedem Sherry,"Cooking Wine, - 12 per case.",12.7,Ounce,None,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49


In [ ]:
def normalize_quantity(df):
    """
    Converts size/unit columns into normalized numeric quantity and standard unit:
    - Weight → grams (g)
    - Volume → litres (L)
    - 'Case' stays as 'case'
    """

    def convert(value, unit):
        if pd.isna(value) or pd.isna(unit):
            return np.nan, np.nan

        try:
            val = float(value)
        except ValueError:
            return np.nan, np.nan

        unit_lower = unit.strip().lower()

        # Weight → grams
        if unit_lower in ['g', 'gram', 'grams']:
            return val, 'g'
        elif unit_lower in ['kg', 'kgs', 'kilogram', 'kilograms']:
            return val * 1000, 'g'
        elif unit_lower in ['oz', 'ounce', 'ounces']:
            return val * 28.3495, 'g'
        elif unit_lower in ['lb', 'lbs', 'pound', 'pounds']:
            return val * 453.592, 'g'

        # Volume → litres
        elif unit_lower in ['l', 'lt', 'litre', 'liter', 'litres', 'liters']:
            return val, 'L'
        elif unit_lower in ['ml', 'mL', 'millilitre', 'milliliter', 'millilitres', 'milliliters']:
            return val / 1000, 'L'
        elif 'fl' in unit_lower and 'oz' in unit_lower:
            return val * 0.0295735, 'L'

        # Case — keep as is
        elif 'case' in unit_lower or 'cases' in unit_lower:
            return val, 'case'

        # Unknown unit
        else:
            return np.nan, np.nan

    df[['Normalized_Quantity', 'Normalized_Unit']] = df.apply(
        lambda row: pd.Series(convert(row['Size_Value'], row['Unit'])), axis=1
    )
    df=df.drop(['Size_Value','Unit'], axis=1)


    return df

In [ ]:
df_extracted=normalize_quantity(df_extracted)

/tmp/ipython-input-2258808902.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Normalized_Quantity', 'Normalized_Unit']] = df.apply(
/tmp/ipython-input-2258808902.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Normalized_Quantity', 'Normalized_Unit']] = df.apply(


In [ ]:
df_extracted.head()

,sample_id,Brand_Name,Product_Name_Clean,Pack_Count,Catalog_Content_Full,image_link,price,Normalized_Quantity,Normalized_Unit
0,33127,La Victoria,Green Taco Sauce Mild,6,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,g
1,198967,Salerno Cookies,The Original Butter Cookies,4,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,g
2,261251,Bear Creek,"Hearty Soup Bowl, Creamy Chicken with Rice",6,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,g
3,55858,Judee,’s Blue Cheese Powder - Gluten-Free and Nut-Fr...,None,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,g
4,292686,kedem Sherry,"Cooking Wine, - 12 per case.",None,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,g


In [ ]:
! pip install sentence-transformers pandas

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
sentences = df_extracted['Catalog_Content_Full'].tolist()
# 3. Generate embeddings for the list of sentences
# The .encode() method returns a NumPy array where each row is an embedding.
catalog_embeddings = model.encode(sentences)

# 4. Display the output
print("Shape of the embeddings array:", catalog_embeddings.shape)
print("\nHere are the embeddings for the 'catalog_content' column:")
print(catalog_embeddings)# 3. Generate embeddings for the list of sentences
# The .encode() method returns a NumPy array where each row is an embedding.
catalog_embeddings = model.encode(sentences)

# 4. Display the output
print("Shape of the embeddings array:", catalog_embeddings.shape)
print("\nHere are the embeddings for the 'catalog_content' column:")
print(catalog_embeddings)

Shape of the embeddings array: (75000, 384)

Here are the embeddings for the 'catalog_content' column:
[[ 0.03326041  0.01084005 -0.05602307 ... -0.05849098  0.00150047
  -0.00965684]
 [-0.0422821   0.03143436 -0.04748075 ... -0.07082944 -0.03443732
   0.02636577]
 [-0.0336338   0.00300811  0.01666682 ... -0.05068669 -0.02866562
   0.01687206]
 ...
 [-0.07520831  0.0302861  -0.02349681 ... -0.0638796  -0.05987212
  -0.02243588]
 [-0.04523224 -0.03537314 -0.01549084 ...  0.00929437  0.05827348
  -0.0643902 ]
 [-0.00743869  0.03607912 -0.05321958 ... -0.100991    0.02066681
   0.07365517]]
Shape of the embeddings array: (75000, 384)

Here are the embeddings for the 'catalog_content' column:
[[ 0.03326041  0.01084005 -0.05602307 ... -0.05849098  0.00150047
  -0.00965684]
 [-0.0422821   0.03143436 -0.04748075 ... -0.07082944 -0.03443732
   0.02636577]
 [-0.0336338   0.00300811  0.01666682 ... -0.05068669 -0.02866562
   0.01687206]
 ...
 [-0.07520831  0.0302861  -0.02349681 ... -0.0638796  

In [ ]:
# We create column names like 'embedding_0', 'embedding_1', etc.
embedding_cols = [f'embedding_{i}' for i in range(catalog_embeddings.shape[1])]
df_embeddings = pd.DataFrame(catalog_embeddings, columns=embedding_cols)

In [ ]:
df_combined = pd.concat([df_extracted, df_embeddings], axis=1)

# 3. Display the final combined DataFrame
print("DataFrame with embeddings combined:")
display(df_combined.head())

DataFrame with embeddings combined:


,sample_id,Brand_Name,Product_Name_Clean,Pack_Count,Catalog_Content_Full,image_link,price,Normalized_Quantity,Normalized_Unit,Brand_Label,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,33127,La Victoria,Green Taco Sauce Mild,6,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,g,14407,...,-0.009825,-0.031103,0.042088,-0.018127,-0.024876,0.017960,0.017674,-0.058491,0.001500,-0.009657
1,198967,Salerno Cookies,The Original Butter Cookies,4,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,g,22931,...,-0.002846,-0.061159,0.005115,0.020050,0.095334,-0.001726,0.098191,-0.070829,-0.034437,0.026366
2,261251,Bear Creek,"Hearty Soup Bowl, Creamy Chicken with Rice",6,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,g,2586,...,0.011707,0.032602,-0.012787,-0.041832,-0.013877,-0.007768,0.032651,-0.050687,-0.028666,0.016872
3,55858,Judee,’s Blue Cheese Powder - Gluten-Free and Nut-Fr...,None,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,g,12922,...,-0.005596,-0.037883,-0.032917,-0.006095,0.108860,-0.048802,-0.020941,-0.063594,0.047193,0.000133
4,292686,kedem Sherry,"Cooking Wine, - 12 per case.",None,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,g,29054,...,-0.014247,0.006413,0.023814,0.029915,-0.003298,-0.039553,0.118118,-0.086265,-0.012085,-0.026015


In [ ]:
df_combined.drop(['Catalog_Content_Full','Product_Name_Clean'], axis=1, inplace=True)

In [ ]:
df_combined.head()

,sample_id,Brand_Name,Pack_Count,image_link,price,Normalized_Quantity,Normalized_Unit,Brand_Label,Unit_Label,embedding_0,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,33127,La Victoria,6,https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,g,14407,2,0.033260,...,-0.009825,-0.031103,0.042088,-0.018127,-0.024876,0.017960,0.017674,-0.058491,0.001500,-0.009657
1,198967,Salerno Cookies,4,https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,g,22931,2,-0.042282,...,-0.002846,-0.061159,0.005115,0.020050,0.095334,-0.001726,0.098191,-0.070829,-0.034437,0.026366
2,261251,Bear Creek,6,https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,g,2586,2,-0.033634,...,0.011707,0.032602,-0.012787,-0.041832,-0.013877,-0.007768,0.032651,-0.050687,-0.028666,0.016872
3,55858,Judee,None,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,g,12922,2,-0.107012,...,-0.005596,-0.037883,-0.032917,-0.006095,0.108860,-0.048802,-0.020941,-0.063594,0.047193,0.000133
4,292686,kedem Sherry,None,https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,g,29054,2,0.054223,...,-0.014247,0.006413,0.023814,0.029915,-0.003298,-0.039553,0.118118,-0.086265,-0.012085,-0.026015


In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Make copies to avoid overwriting original data
df_extracted['Brand_Label'] = LabelEncoder().fit_transform(df_extracted['Brand_Name'].astype(str))
df_extracted['Unit_Label'] = LabelEncoder().fit_transform(df_extracted['Normalized_Unit'].astype(str))


In [ ]:
df_extracted.head()

,sample_id,Brand_Name,Product_Name_Clean,Pack_Count,Catalog_Content_Full,image_link,price,Normalized_Quantity,Normalized_Unit,Brand_Label,Unit_Label
0,33127,La Victoria,Green Taco Sauce Mild,6,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,g,14407,2
1,198967,Salerno Cookies,The Original Butter Cookies,4,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,g,22931,2
2,261251,Bear Creek,"Hearty Soup Bowl, Creamy Chicken with Rice",6,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,g,2586,2
3,55858,Judee,’s Blue Cheese Powder - Gluten-Free and Nut-Fr...,None,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,g,12922,2
4,292686,kedem Sherry,"Cooking Wine, - 12 per case.",None,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,g,29054,2


In [ ]:
Brand_Encoder=LabelEncoder()
df_combined['Brand_Label'] = Brand_Encoder.fit_transform(df_extracted['Brand_Name'].astype(str))
Unit_Encoder=LabelEncoder()
df_combined['Unit_Label'] = Unit_Encoder.fit_transform(df_extracted['Normalized_Unit'].astype(str))



In [ ]:
import pickle
with open('Brand_Encoder.pkl', 'wb') as f:
    pickle.dump(Brand_Encoder, f)
with open('Unit_Encoder.pkl', 'wb') as f:
    pickle.dump(Unit_Encoder, f)

In [ ]:
df_combined.head()

,sample_id,Brand_Name,Pack_Count,image_link,price,Normalized_Quantity,Normalized_Unit,Brand_Label,Unit_Label,embedding_0,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,33127,La Victoria,6,https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,g,14407,2,0.033260,...,-0.009825,-0.031103,0.042088,-0.018127,-0.024876,0.017960,0.017674,-0.058491,0.001500,-0.009657
1,198967,Salerno Cookies,4,https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,g,22931,2,-0.042282,...,-0.002846,-0.061159,0.005115,0.020050,0.095334,-0.001726,0.098191,-0.070829,-0.034437,0.026366
2,261251,Bear Creek,6,https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,g,2586,2,-0.033634,...,0.011707,0.032602,-0.012787,-0.041832,-0.013877,-0.007768,0.032651,-0.050687,-0.028666,0.016872
3,55858,Judee,None,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,g,12922,2,-0.107012,...,-0.005596,-0.037883,-0.032917,-0.006095,0.108860,-0.048802,-0.020941,-0.063594,0.047193,0.000133
4,292686,kedem Sherry,None,https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,g,29054,2,0.054223,...,-0.014247,0.006413,0.023814,0.029915,-0.003298,-0.039553,0.118118,-0.086265,-0.012085,-0.026015


In [ ]:
df_combined.drop(['Brand_Name','Normalized_Unit'], axis=1, inplace=True)

In [ ]:
df_combined.head()

,sample_id,Pack_Count,image_link,price,Normalized_Quantity,Brand_Label,Unit_Label,embedding_0,embedding_1,embedding_2,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,33127,6,https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,14407,2,0.033260,0.010840,-0.056023,...,-0.009825,-0.031103,0.042088,-0.018127,-0.024876,0.017960,0.017674,-0.058491,0.001500,-0.009657
1,198967,4,https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,22931,2,-0.042282,0.031434,-0.047481,...,-0.002846,-0.061159,0.005115,0.020050,0.095334,-0.001726,0.098191,-0.070829,-0.034437,0.026366
2,261251,6,https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,2586,2,-0.033634,0.003008,0.016667,...,0.011707,0.032602,-0.012787,-0.041832,-0.013877,-0.007768,0.032651,-0.050687,-0.028666,0.016872
3,55858,None,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,12922,2,-0.107012,-0.079453,0.012586,...,-0.005596,-0.037883,-0.032917,-0.006095,0.108860,-0.048802,-0.020941,-0.063594,0.047193,0.000133
4,292686,None,https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,29054,2,0.054223,0.072425,-0.043134,...,-0.014247,0.006413,0.023814,0.029915,-0.003298,-0.039553,0.118118,-0.086265,-0.012085,-0.026015


In [ ]:
# Save DataFrame to CSV
df_combined.to_csv('df_conbined_processed.csv', index=False)


In [ ]:
from google.colab import files

files.download('df_conbined_processed.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
! pip install tensorflow Pillow requests xgboost scikit-learn


In [ ]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import os
from google.colab import drive

# --- 0. Mount Google Drive ---
drive.mount('/content/drive')
save_dir = '/content/drive/MyDrive/image_embeddings_batches'
os.makedirs(save_dir, exist_ok=True)

# --- 1. Input DataFrame ---
# df_combined should already exist with columns like:
# 'sample_id', 'Brand_Name', 'Product_Description', 'image_link', 'Brand_Label', 'Unit_Label', etc.

# 2. Setup pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# 3. Function to generate image embedding
def get_embedding_from_url_tf(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert('RGB').resize((224, 224))
        img_array = image.img_to_array(img)
        img_array_expanded = np.expand_dims(img_array, axis=0)
        img_preprocessed = preprocess_input(img_array_expanded)
        embedding = base_model.predict(img_preprocessed, verbose=0)
        return embedding.flatten()
    except Exception as e:
        # Return zero vector if image fails
        return np.zeros((2048,))

# 4. Function to process a batch and save
def process_and_save_batch(df_batch, batch_index, save_dir=save_dir):
    tqdm.pandas(desc=f"Processing Batch {batch_index}")
    embedding_list = df_batch['image_link'].progress_apply(get_embedding_from_url_tf).tolist()

    # Convert embeddings list to DataFrame
    df_embeddings = pd.DataFrame(embedding_list)
    df_embeddings.columns = [f'image_emb_{i}' for i in range(df_embeddings.shape[1])]

    # Reset index and concat
    df_batch = df_batch.reset_index(drop=True)
    df_embeddings = df_embeddings.reset_index(drop=True)
    df_combined_batch = pd.concat([df_batch, df_embeddings], axis=1)

    # Save batch to Drive
    batch_file = os.path.join(save_dir, f'batch_{batch_index}_embeddings.csv')
    df_combined_batch.to_csv(batch_file, index=False)
    print(f"Saved Batch {batch_index} to {batch_file}")

    return df_combined_batch

# 5. Process df_combined in 20% chunks
total_rows = len(df_combined)
chunk_size = int(total_rows * 0.2)
dfs = []

for batch_index, start in enumerate(range(0, total_rows, chunk_size), 1):
    end = min(start + chunk_size, total_rows)
    df_batch = df_combined.iloc[start:end].copy()
    print(f"\nProcessing rows {start} to {end}")
    df_combined_batch = process_and_save_batch(df_batch, batch_index)
    dfs.append(df_combined_batch)

# 6. Merge all batches into final DataFrame
df_final = pd.concat(dfs, axis=0).reset_index(drop=True)

# 7. Save final DataFrame to Google Drive
final_csv_path = '/content/drive/MyDrive/catalog_with_image_embeddings.csv'
df_final.to_csv(final_csv_path, index=False)
print(f"\nAll batches merged. Final CSV saved to {final_csv_path}")

# 8. df_final is now ready for ensemble regression


KeyboardInterrupt: 

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

save_dir = '/content/drive/MyDrive/image_embeddings_batches'
existing_batches = [f for f in os.listdir(save_dir) if f.endswith('.csv')]
existing_batches.sort()
print("Existing batches:", existing_batches)


Existing batches: ['batch_1_embeddings.csv', 'batch_2_embeddings.csv', 'batch_3_embeddings.csv']


In [ ]:
total_rows = len(df_combined)
chunk_size = int(total_rows * 0.2)

# Determine the starting batch index
start_batch_index = len(existing_batches) + 1

dfs = []

for batch_index, start in enumerate(range(0, total_rows, chunk_size), 1):
    if batch_index < start_batch_index:
        print(f"Skipping already processed Batch {batch_index}")
        # Optionally, load the saved batch
        batch_file = os.path.join(save_dir, f'batch_{batch_index}_embeddings.csv')
        dfs.append(pd.read_csv(batch_file))
        continue

    end = min(start + chunk_size, total_rows)
    df_batch = df_combined.iloc[start:end].copy()
    print(f"\nProcessing rows {start} to {end} (Batch {batch_index})")
    df_combined_batch = process_and_save_batch(df_batch, batch_index)
    dfs.append(df_combined_batch)


Skipping already processed Batch 1
Skipping already processed Batch 2
Skipping already processed Batch 3

Processing rows 45000 to 60000 (Batch 4)


Processing Batch 4: 100%|██████████| 15000/15000 [29:56<00:00,  8.35it/s]


Saved Batch 4 to /content/drive/MyDrive/image_embeddings_batches/batch_4_embeddings.csv

Processing rows 60000 to 75000 (Batch 5)


Processing Batch 5: 100%|██████████| 15000/15000 [29:58<00:00,  8.34it/s]


Saved Batch 5 to /content/drive/MyDrive/image_embeddings_batches/batch_5_embeddings.csv


df_final = pd.concat(dfs, axis=0).reset_index(drop=True)
final_csv_path = '/content/drive/MyDrive/catalog_with_image_embeddings.csv'
df_final.to_csv(final_csv_path, index=False)
print(f"\nAll batches merged. Final CSV saved to {final_csv_path}")


In [ ]:
df_combined=pd.read_csv('df_conbined_processed.csv')

In [ ]:
df_combined.shape

(75000, 391)

In [ ]:
def get_embedding_from_url_tf(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert('RGB').resize((224, 224))
        img_array = image.img_to_array(img)
        img_array_expanded = np.expand_dims(img_array, axis=0)
        img_preprocessed = preprocess_input(img_array_expanded)
        embedding = base_model.predict(img_preprocessed, verbose=0)
        return embedding.flatten()
    except Exception as e:
        # Return zero vector if image fails
        return np.zeros((2048,))

def process_and_save_batch(df_batch, batch_index, save_dir=save_dir):
    tqdm.pandas(desc=f"Processing Batch {batch_index}")
    embedding_list = df_batch['image_link'].progress_apply(get_embedding_from_url_tf).tolist()

    # Convert embeddings list to DataFrame
    df_embeddings = pd.DataFrame(embedding_list)
    df_embeddings.columns = [f'image_emb_{i}' for i in range(df_embeddings.shape[1])]

    # Reset index and concat
    df_batch = df_batch.reset_index(drop=True)
    df_embeddings = df_embeddings.reset_index(drop=True)
    df_combined_batch = pd.concat([df_batch, df_embeddings], axis=1)

    # Save batch to Drive
    batch_file = os.path.join(save_dir, f'batch_{batch_index}_embeddings.csv')
    df_combined_batch.to_csv(batch_file, index=False)
    print(f"Saved Batch {batch_index} to {batch_file}")

    return df_combined_batch

In [ ]:
dfs

[       sample_id  Pack_Count  \
 0          33127         6.0   
 1         198967         4.0   
 2         261251         6.0   
 3          55858         NaN   
 4         292686         NaN   
 ...          ...         ...   
 14995      33362         NaN   
 14996      81531         4.0   
 14997     208579         1.0   
 14998      20819         2.0   
 14999     185483         NaN   
 
                                               image_link  price  \
 0      https://m.media-amazon.com/images/I/51mo8htwTH...   4.89   
 1      https://m.media-amazon.com/images/I/71YtriIHAA...  13.12   
 2      https://m.media-amazon.com/images/I/51+PFEe-w-...   1.97   
 3      https://m.media-amazon.com/images/I/41mu0HAToD...  30.34   
 4      https://m.media-amazon.com/images/I/41sA037+Qv...  66.49   
 ...                                                  ...    ...   
 14995  https://m.media-amazon.com/images/I/618ByvX4Ak...  53.54   
 14996  https://m.media-amazon.com/images/I/81XxDkvl8q... 

In [ ]:
df_final = pd.concat(dfs, axis=0).reset_index(drop=True)
final_csv_path = '/content/drive/MyDrive/catalog_with_image_embeddings.csv'
df_final.to_csv(final_csv_path, index=False)
print(f"\nAll batches merged. Final CSV saved to {final_csv_path}")



All batches merged. Final CSV saved to /content/drive/MyDrive/catalog_with_image_embeddings.csv


In [ ]:
df_final.to_csv('df_final_resnet.csv', index=False)

In [ ]:
from google.colab import files

files.download('df_final_resnet.csv')

FileNotFoundError: Cannot find file: df_final_resnet.csv

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
final_csv_path = '/content/drive/MyDrive/catalog_with_image_embeddings.csv'
df_final = pd.read_csv(final_csv_path)
df_final.to_csv('df_final_resnet.csv', index=False)

In [ ]:
from google.colab import files
files.download('df_final_resnet.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##PCA

In [ ]:
import pandas as pd
df_final_resnet=pd.read_csv('df_final_resnet.csv')

In [ ]:
df_final_resnet.head()
df_final_resnet.fillna({'Pack_Count': 1, 'Normalized_Quantity': 1, }, inplace=True)

In [ ]:
df_final_resnet.head()

,sample_id,Pack_Count,image_link,price,Normalized_Quantity,Brand_Label,Unit_Label,embedding_0,embedding_1,embedding_2,...,image_emb_2038,image_emb_2039,image_emb_2040,image_emb_2041,image_emb_2042,image_emb_2043,image_emb_2044,image_emb_2045,image_emb_2046,image_emb_2047
0,33127,6.0,https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,14407,2,0.033260,0.010840,-0.056023,...,0.328564,0.059449,0.010244,0.000000,0.000000,0.561996,0.000000,0.003054,0.551566,0.071454
1,198967,4.0,https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,22931,2,-0.042282,0.031434,-0.047481,...,0.248163,0.000000,0.095125,0.000000,0.000000,0.000000,0.573756,0.000000,0.448992,0.172074
2,261251,6.0,https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,2586,2,-0.033634,0.003008,0.016667,...,0.158004,0.197852,0.007659,0.000000,0.003244,0.189149,0.678307,0.162803,0.288635,0.550170
3,55858,1.0,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,12922,2,-0.107012,-0.079453,0.012586,...,0.000000,0.000000,0.107039,0.030175,0.033478,0.144659,0.976481,0.797653,0.175869,0.384599
4,292686,1.0,https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,29054,2,0.054223,0.072425,-0.043134,...,0.000000,0.007327,0.006289,0.003398,0.178672,1.108397,0.000000,0.000000,0.147693,0.023560


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle

# --- 1. Select only the feature columns for PCA ---
# Assuming df_final contains embeddings and other columns like sample_id, Brand_Label, etc.
# Keep only numeric embedding columns
embedding_cols = [col for col in df_final_resnet.columns if col.startswith('embedding_') or col.startswith('image_emb_') or col.startswith('Normalized_Quantity')]
df_features = df_final_resnet[embedding_cols]

print(f"Number of features before PCA: {df_features.shape[1]}")

# --- 2. Standardize features (important for PCA) ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_features)

# --- 3. Fit PCA to retain 95% variance ---
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"Reduced dimensions after PCA: {X_pca.shape[1]}")

# --- 4. Convert PCA result back to DataFrame ---
df_pca = pd.DataFrame(X_pca, columns=[f'pca_{i}' for i in range(X_pca.shape[1])])
df_pca.index = df_final_resnet.index  # Keep original indexing

# --- 5. Combine with other non-feature columns if needed ---
non_feature_cols = [col for col in df_final_resnet.columns if col not in embedding_cols]
df_final_pca = pd.concat([df_final_resnet[non_feature_cols].reset_index(drop=True), df_pca.reset_index(drop=True)], axis=1)

# --- 6. Save the transformed DataFrame ---
df_final_pca.to_csv('/content/drive/MyDrive/df_final_pca_95.csv', index=False)
df_final_pca.to_csv('df_final_pca', index=False)
print("PCA-transformed DataFrame saved to Google Drive.")

# --- 7. Save PCA and scaler for future use ---
with open('/content/drive/MyDrive/pca_model_95.pkl', 'wb') as f:
    pickle.dump(pca_res, f)

with open('/content/drive/MyDrive/scaler_95.pkl', 'wb') as f:
    pickle.dump(scaler_res, f)

print("PCA model and scaler saved to Google Drive.")


Number of features before PCA: 2433
Reduced dimensions after PCA: 1281
PCA-transformed DataFrame saved to Google Drive.


NameError: name 'pca_res' is not defined

In [ ]:
# --- 7. Save PCA and scaler for future use ---
with open('pca_res.pkl', 'wb') as f:
    pickle.dump(pca, f)

with open('scaler_res.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("PCA model and scaler saved to Google Drive.")

PCA model and scaler saved to Google Drive.


In [ ]:
with open('pca_res.pkl', 'wb') as f:
    pickle.dump(pca, f)

In [ ]:
from google.colab import files
files.download('pca_res.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import pandas as pd
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / denominator) * 100  # percentage


In [13]:
df_final=pd.read_csv('df_final_clip.csv')

In [16]:
df_final.fillna({'Pack_Count': 1, 'Normalized_Quantity': 1, }, inplace=True)
df_final.head()

,sample_id,Pack_Count,image_link,price,Normalized_Quantity,Brand_Label,Unit_Label,embedding_0,embedding_1,embedding_2,...,clip_pt_img_emb_502,clip_pt_img_emb_503,clip_pt_img_emb_504,clip_pt_img_emb_505,clip_pt_img_emb_506,clip_pt_img_emb_507,clip_pt_img_emb_508,clip_pt_img_emb_509,clip_pt_img_emb_510,clip_pt_img_emb_511
0,33127,6.0,https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,340.194000,14407,2,0.033260,0.010840,-0.056023,...,-0.016605,0.381696,-0.245876,0.190976,0.239461,0.140391,0.077045,0.462531,0.081503,-0.364598
1,198967,4.0,https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,226.796000,22931,2,-0.042282,0.031434,-0.047481,...,-0.472628,0.266820,-0.443731,0.230982,-0.063851,-0.337085,0.034931,0.407764,-0.417217,0.037924
2,261251,6.0,https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,53.864050,2586,2,-0.033634,0.003008,0.016667,...,0.147087,0.150716,-0.227894,0.782844,-0.529392,0.041528,-0.093397,0.293706,-0.248547,-0.213709
3,55858,1.0,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,318.931875,12922,2,-0.107012,-0.079453,0.012586,...,0.196246,0.166772,-0.083480,0.137943,-0.097288,0.386728,-0.318723,0.265707,-0.538017,0.169577
4,292686,1.0,https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,360.038650,29054,2,0.054223,0.072425,-0.043134,...,-0.017257,-0.136049,-0.118537,-0.310866,0.133500,0.145094,-0.435640,0.800807,0.240080,-0.417644


In [11]:
# --- 1. Imports ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.metrics import make_scorer
import optuna

In [36]:
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return 100*np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))

# XGBoost-compatible SMAPE
def smape_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    return 'smape', smape(y_true, y_pred)

In [37]:
df_test = df_final.copy()

In [38]:
target_col = 'price'

# Drop any irrelevant columns (keep embeddings + numeric features)
# Adjust according to your df_final column names
drop_cols = ['sample_id', 'image_link', 'price']
X = df_test.drop(columns=drop_cols)
y = df_test[target_col]

# --- 4. Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- 5. Standard scaling ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [39]:
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Number of features before PCA: {X_train.shape[1]}")
print(f"Reduced dimensions after PCA: {X_train_pca.shape[1]}")

Number of features before PCA: 900
Reduced dimensions after PCA: 486


In [40]:
X.shape

(75000, 900)

In [41]:
from tqdm import tqdm

# --- 1. Create a progress bar ---
n_trials = 10  # Or however many you want
pbar = tqdm(total=n_trials, desc="Optuna Trials")

# --- 2. Define callback to update progress bar ---
def tqdm_callback(study, trial):
    pbar.update(1)


# --- 8. Optuna objective ---
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 200, 2000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    model = xgb.XGBRegressor(**params, n_jobs=-1)

    # --- Fit without eval_metric or eval_set ---
    model.fit(X_train_pca, y_train)

    preds = model.predict(X_test_pca)
    return smape(y_test, preds)


# --- 9. Run Optuna study ---
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10,callbacks=[tqdm_callback])  # Reduce trials for quick test

Optuna Trials: 100%|██████████| 10/10 [1:20:06<00:00, 480.63s/it]
[I 2025-10-13 02:31:40,890] A new study created in memory with name: no-name-6b775911-2f8a-452f-8187-578c9fc75144
/tmp/ipython-input-1854363488.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
/tmp/ipython-input-1854363488.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
/tmp/ipython-input-1854363488.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float inste

KeyboardInterrupt: 

In [42]:
print("Best SMAPE on 30k subset:", study.best_value)
print("Best hyperparameters:", study.best_params)



Best SMAPE on 30k subset: 66.17775040621368
Best hyperparameters: {'learning_rate': 0.014283608016225975, 'max_depth': 12, 'subsample': 0.8988937699566887, 'colsample_bytree': 0.7295015460703748, 'n_estimators': 1200, 'reg_alpha': 0.000241095280571718, 'reg_lambda': 8.647414052717895e-07, 'min_child_weight': 7}


In [58]:
# --- 10. Train final XGBoost model ---
best_params = study.best_params
final_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', n_jobs=-1)
final_model.fit(X_train_pca, y_train)

# --- 11. Evaluate ---
y_pred = final_model.predict(X_test_pca)
print("Final SMAPE on test set:", smape(y_test, y_pred))

Final SMAPE on test set: 66.17775040621368


KeyError: 1

In [48]:
X.head()

,Pack_Count,Normalized_Quantity,Brand_Label,Unit_Label,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,...,clip_pt_img_emb_502,clip_pt_img_emb_503,clip_pt_img_emb_504,clip_pt_img_emb_505,clip_pt_img_emb_506,clip_pt_img_emb_507,clip_pt_img_emb_508,clip_pt_img_emb_509,clip_pt_img_emb_510,clip_pt_img_emb_511
0,6.0,340.194000,14407,2,0.033260,0.010840,-0.056023,0.088746,-0.000599,0.023403,...,-0.016605,0.381696,-0.245876,0.190976,0.239461,0.140391,0.077045,0.462531,0.081503,-0.364598
1,4.0,226.796000,22931,2,-0.042282,0.031434,-0.047481,0.068237,0.012065,-0.059747,...,-0.472628,0.266820,-0.443731,0.230982,-0.063851,-0.337085,0.034931,0.407764,-0.417217,0.037924
2,6.0,53.864050,2586,2,-0.033634,0.003008,0.016667,0.083491,0.001591,0.052957,...,0.147087,0.150716,-0.227894,0.782844,-0.529392,0.041528,-0.093397,0.293706,-0.248547,-0.213709
3,1.0,318.931875,12922,2,-0.107012,-0.079453,0.012586,-0.057814,0.087116,0.024524,...,0.196246,0.166772,-0.083480,0.137943,-0.097288,0.386728,-0.318723,0.265707,-0.538017,0.169577
4,1.0,360.038650,29054,2,0.054223,0.072425,-0.043134,0.051806,-0.111483,0.008939,...,-0.017257,-0.136049,-0.118537,-0.310866,0.133500,0.145094,-0.435640,0.800807,0.240080,-0.417644


In [53]:
X_test_scaled.shape


(15000, 900)

In [54]:
final_model.predict(X.loc[378:378])

ValueError: Feature shape mismatch, expected: 486, got 900

In [56]:
final_model.save_model("final_model_1.json")   # or .bin


In [57]:
import joblib
joblib.dump(scaler, "scaler_1.pkl")
joblib.dump(pca, "pca_1.pkl")

['pca_1.pkl']